## Parameter Fitting to a Simple Series Reaction

jbr book reference: [Figure A.10](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html)

Reaction is $A \xrightarrow[]{k_1} B \xrightarrow[]{k_2} C$

Data simulated for $A$, $B$, and $C$ over time (with added noise) based on:
$ c_{A0} = 1, c_{B0} = 0, c_{c0} = 0, k_1 = 2, k_2 = 1$

The above parameters are also the parameters that will be fitted based on the data

In [1]:
import kipet
import numpy as np
import pyomo.environ as pyo
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import t

In [2]:
dirname = '/home/paperspace/learn_kipet/kipet_paresto/data_sets/'
filename = dirname + 'ABC_data.csv'
C_data = pd.read_csv(filename, index_col = 0)

In [3]:
C_data.columns = ['A', 'B', 'C']
C_data.head()

,A,B,C
t,,,
0.000,0.957,-0.031,-0.015
0.263,0.557,0.330,0.044
0.526,0.342,0.512,0.156
0.789,0.224,0.499,0.310
1.053,0.123,0.428,0.454


In [53]:
r1 = kipet.ReactionModel('rxn1')

A = r1.component('A', value = 1.0)
B = r1.component('B', value = 0.0)
C = r1.component('C', value = 0.0)

k1 = r1.parameter('k1',value = 5.0, bounds=(0.0,10.0), fixed = False)
k2 = r1.parameter('k2',value = 5.0, bounds=(0.0,10.0), fixed = False)

In [54]:
r1.add_ode('A', -k1 * A)
r1.add_ode('B', k1 * A - k2 * B)
r1.add_ode('C', k2 * B)

In [55]:
r1.add_data(data = C_data)

In [56]:
#r1.settings

In [57]:
#r1.settings.collocation.nfe = 50
#r1.settings.collocation.ncp = 1
#r1.settings.collocation.scheme = 'LAGRANGE-RADAU'
#r1.settings.parameter_estimator.solver = 'ipopt'
r1.settings.solver.linear_solver = 'ma27'
r1.settings.simulator.tee = True

In [58]:
r1.run_opt()

########################################
# KIPET version 1.0.6
# Date: 2021-09-23-19-59-34
# File: <ipython-input-53-7a8fc719098a>
# ReactionModel instance: rxn1
########################################

# Simulator: Initializing with starting values
# Simulator: Using the fe method
# Simulator: Setting up simulation model
Warning! Since Cm-matrix contains negative values Kipet is assuming a derivative of Cm has been inputted
Warning! Since Cm-matrix contains negative values Kipet is assuming a derivative of Cm has been inputted
Warning! Since Cm-matrix contains negative values Kipet is assuming a derivative of Cm has been inputted
# TemplateBuilder: Preparing model for simulator
# Simulator: Finished creating simulator
Ipopt 3.13.4: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more

Exception: The current iteration was unsuccessful.

In [ ]:
r1.results.show_parameters

In [ ]:
fig, ax = plt.subplots()
for c in ['A', 'B', 'C']:
    ax.scatter(r1.results.Cm.index, r1.results.Cm[c])
    ax.plot(r1.results.Z.index, r1.results.Z[c])

In [ ]:
results_pyomo.objective

In [ ]:
# since kipet outputs reduced hessian which equals covariance if right sigma values are specified. 
# since we just specified sigma=1, we need to adjust the covariance matrix output from KIPET to get the 
# estimated covariance matrix
mse = r1.results.objective / (C_data.shape[0]*3 - 5)
cov_est = 2 * mse * r1.results.parameter_covariance
cov_est

In [ ]:
dof = (C_data.shape[0]*3 - 5)
conf_delta = t.ppf(0.975, dof) * np.sqrt(np.diag(cov_est))
conf_delta

In [ ]:
# confidence limits of parameters
i = 0
for k, v in r1.results.P.items():
    print(f"{k}, {v - conf_delta[i]:.2f}, {v + conf_delta[i]:.2f}")
    i = i + 1